<a href="https://colab.research.google.com/github/gupta24789/sentiment-analysis/blob/main/sentiment_rnn_lighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q lightning
!pip install -q neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import neattext as nt
import itertools
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.nn.utils.rnn import  pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset

from torchmetrics import Accuracy
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

warnings.filterwarnings('ignore')

In [6]:
train_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/train.csv")
val_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/val.csv")

train_df = train_df[['raw_tweet', 'label']].dropna().reset_index(drop = True)
val_df = val_df[['raw_tweet', 'label']].dropna().reset_index(drop = True)

In [7]:
## Clean the data
def custom_clean_text(x):
  x = nt.TextFrame(x)
  x = x.remove_stopwords().remove_urls().remove_emails().remove_dates().remove_puncts().remove_numbers().remove_userhandles().remove_multiple_spaces()
  x = x.text.lower()
  return x

train_df['processed_text'] = train_df.raw_tweet.apply(lambda x: custom_clean_text(x))
val_df['processed_text'] = val_df.raw_tweet.apply(lambda x: custom_clean_text(x))

In [8]:
train_df.head(4)

,raw_tweet,label,processed_text
0,Want to say a huge thanks to @WarriorAssaultS ...,1.0,want huge thanks #ff thanks support :)
1,@jaynehh_ you just need a job and get a letter...,1.0,need job letter work place saying work letter...
2,"@knhillrocks HA yes, make it quick tho :D",1.0,ha yes quick tho :d
3,@shartyboy Thanks for texting me back :)) I'm ...,1.0,thanks texting :)) im texting tomorrow :))


In [9]:
X_train = train_df.processed_text
y_train = train_df.label

X_val = val_df.processed_text
y_val = val_df.label

## Create Vocab

In [10]:
special_tokens = ['__PAD__','__UNK__']
words = list(set(itertools.chain.from_iterable(train_df.processed_text.apply(lambda x: x.split(" ")))))
words = special_tokens +  words
token2idx = {w:i for i,w in enumerate(words)}
idx2tokens = {i:w for i,w in enumerate(words)}
print(f"vocab size : {len(token2idx)}")

vocab size : 11332


## Convert words to numbers

In [11]:
def convert_token_to_number(tweet, verbose = False):
  unk_token_id = token2idx['__UNK__']
  encoded_tweet = []

  if verbose:
    print(f"UNK TOKEN ID : {unk_token_id}")
    print(f"RAW TWEET : {tweet}")

  for w in tweet.split(" "):
    if w in token2idx:
      encoded_tweet.append(token2idx[w])
    else:
      encoded_tweet.append(unk_token_id)

  return encoded_tweet

In [12]:
X_train_encoded = X_train.apply(lambda x: convert_token_to_number(x))
X_val_encoded = X_val.apply(lambda x: convert_token_to_number(x))

In [13]:
def data_collator(batch):
  features = [torch.tensor(item[0]) for item in batch]
  labels = torch.tensor([item[1] for item in batch], dtype = torch.float32)

  features = pad_sequence(features, batch_first=True, padding_value= token2idx['__PAD__'])

  return (features, labels)

In [14]:
batch_size = 32
train_dl = DataLoader(list(zip(X_train_encoded, y_train)), batch_size = batch_size, shuffle = True, collate_fn = data_collator)
val_dl = DataLoader(list(zip(X_val_encoded, y_val)), batch_size = batch_size, shuffle = False, collate_fn = data_collator)

In [15]:
example = next(iter(train_dl))
features, labels = example[0], example[1]
features.shape, labels.shape

(torch.Size([32, 14]), torch.Size([32]))

## RNN

The RNN returns 2 tensors
  - output
  - hidden

```  
output shape : [batch size, sentence length, hidden dim]
hidden shape : [batch size, num_of_rnn_layers, hidden dim]
```

output is the concatenation of the hidden state from every time step, whereas hidden is simply the final hidden state.

In [16]:
class RNNModel(L.LightningModule):

  def __init__(self, vocab_size , emb_dim, hidden_dim, num_classes, learning_rate, dropout, num_layers, is_bidirectional = True):
    super().__init__()
    ## define variable
    self.learning_rate = learning_rate

    ## define layers
    self.embedding = nn.Embedding(num_embeddings= vocab_size, embedding_dim= emb_dim)
    if is_bidirectional:
      self.rnn = nn.RNN(emb_dim, hidden_dim // 2,batch_first=True, dropout = dropout, num_layers = num_layers, bidirectional= is_bidirectional)
    else:
      self.rnn = nn.RNN(emb_dim, hidden_dim,batch_first=True, dropout = dropout, num_layers = num_layers, bidirectional= is_bidirectional)

    self.linear = nn.Linear(hidden_dim, num_classes)
    self.sigmoid = nn.Sigmoid()


    ## loss & metrics
    self.loss_fn = nn.BCELoss()
    self.train_accuracy = Accuracy(task = 'binary', num_classes= 2, threshold = 0.5)
    self.val_accuracy = Accuracy(task = 'binary', num_classes = 2, threshold= 0.5)



  def forward(self, features, verbose = False):
    """
    """
    emb_out = self.embedding(features)
    out, hidden = self.rnn(emb_out)
    squeezed_out = torch.mean(out, dim =1)
    linear_out = self.linear(squeezed_out)
    sigmoid_out = self.sigmoid(linear_out)

    if verbose:
      print(f"Input shape : {features.shape}")
      print(f"EMB shape : {emb_out.shape}")
      print(f"RNN out shape : {out.shape}")
      print(f"RNN hidden shape : {hidden.shape}")
      print(f"RNN squeezed_out shape : {squeezed_out.shape}")
      print(f"linear_out shape : {linear_out.shape}")
      print(f"sigmoid_out shape : {sigmoid_out.shape}")

    return sigmoid_out


  def training_step(self, batch, batch_idx):
    features, labels = batch[0], batch[1]
    logits = self(features)
    logits = logits.squeeze(dim=1)
    loss = self.loss_fn(logits, labels)
    self.train_accuracy(logits, labels)
    self.log_dict({"train_loss": loss, "train_acc": self.train_accuracy}, on_step = False, on_epoch = True, prog_bar = True)

  def validation_step(self,batch, batch_idx):
    features, labels = batch[0], batch[1]
    logits = self(features)
    logits = logits.squeeze(dim=1)
    loss = self.loss_fn(logits, labels)
    self.val_accuracy(logits, labels)
    self.log_dict({"val_loss": loss, "val_acc":  self.val_accuracy}, on_step = False, on_epoch = True, prog_bar = True)

  def on_train_epoch_end(self):
    self.train_accuracy.reset()

  def on_validation_epoch_end(self):
    if self.current_epoch!=0:
      print(f"Epoch : {self.current_epoch} val accuracy : {self.val_accuracy.compute()}")
    self.val_accuracy.reset()

  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

In [17]:
## Test the model
model = RNNModel(vocab_size=len(token2idx),emb_dim =100, hidden_dim = 64,
                 num_classes = 1, learning_rate = .001, dropout = 0.2,
                 num_layers = 1, is_bidirectional = False)

logits = model(features, verbose = True)
logits = logits.squeeze(dim=1)
loss = model.loss_fn(logits, labels)
print(f"\nLoss: {loss}")

Input shape : torch.Size([32, 14])
EMB shape : torch.Size([32, 14, 100])
RNN out shape : torch.Size([32, 14, 64])
RNN hidden shape : torch.Size([1, 32, 64])
RNN squeezed_out shape : torch.Size([32, 64])
linear_out shape : torch.Size([32, 1])
sigmoid_out shape : torch.Size([32, 1])

Loss: 0.6915666460990906


In [20]:
# ## Build Trainer
# model = RNNModel(vocab_size=len(token2idx),emb_dim =100, hidden_dim = 512,
#                  num_classes = 1, learning_rate = .001, dropout = 0.25,
#                  num_layers = 1, is_bidirectional=False)

# ## logger
# logger =  CSVLogger("logs", name="sentiment_analysis")

# ## checkpoints
# checkpoint_callback  = ModelCheckpoint(filename='{epoch}-{val_loss:.2f}-{val_accuracy:.2f}',
#                                         every_n_epochs = 10,
#                                         save_top_k = -1,
#                                         monitor='val_loss',
#                                       )


# trainer = L.Trainer(accelerator="cpu",
#                      max_epochs = 5,
#                      check_val_every_n_epoch=1,
#                      callbacks=[checkpoint_callback],
#                      logger=logger

#                     )

# ## Train the Model
# trainer.fit(model, train_dl, val_dl)